In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from collections import Counter
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from xgboost import plot_tree
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder,normalize
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import xgboost
import inspect
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
import warnings
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#load the dataset
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
df.head(3)

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0


In [3]:
#Store the Id in a dataframe for submission
test_ids = test['Id']

In [4]:
#For the categorical columns ('Id' and 'EJ'), we use the 'fit_transform' method of the 'LabelEncoder' to transform the data.
encoder = LabelEncoder()
df['Id'] = encoder.fit_transform(df['Id']).astype(float)
test['Id'] = encoder.fit_transform(test['Id']).astype(float)

df['EJ'] = encoder.fit_transform(df['EJ']).astype(float)
test['EJ'] = encoder.fit_transform(test['EJ']).astype(float)

KNNImputer replaces missing values with the mean of the 'k' nearest neighbors in the feature space. 

In [5]:
# First, we create an instance of the 'KNNImputer' class.'n_neighbors=5', meaning the algorithm will use the 5 nearest neighbors to impute missing values.
imputer = KNNImputer(n_neighbors=5)

# Next, we extract the column names from the DataFrames for later use
df_col = df.columns
test_col = test.columns

# We use the 'fit_transform' method of the 'KNNImputer' to impute missing values in each DataFrame.
# The 'fit_transform' method returns a new array with the imputed values.
df_imputed = imputer.fit_transform(df)
test_imputed = imputer.fit_transform(test)

# Finally, we create new DataFrames using the imputed values and assign the original column names to them
df = pd.DataFrame(df_imputed, columns=df_col)
test = pd.DataFrame(test_imputed, columns=test_col)

In [6]:
#state the random to a value
state= 0

In [7]:
#Split the data for training removing the target and set the test size to 20%.

X = df.drop(columns = {"Class"})
y = df.pop('Class')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=state)

In [8]:
# Initialize classifiers

rf_classifier = RandomForestClassifier(max_depth = None, min_samples_split = 2, n_estimators=100, random_state=state)
gb_classifier = GradientBoostingClassifier(learning_rate = 0.1, max_depth= 1, n_estimators=300, random_state=state)
svm_classifier = SVC(kernel='rbf', probability=True, random_state=state)
xgb_classifier = XGBClassifier(colsample_bylevel = 0.3, 
                      colsample_bytree = 1.0, 
                      learning_rate = 0.3, 
                      max_depth = 2, 
                      n_estimators = 250, 
                      subsample = 1.0,   
                      scale_pos_weight = 4.943)

In [9]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define classifiers
rf_classifier = RandomForestClassifier(random_state=state)
gb_classifier = GradientBoostingClassifier(random_state=state)
xgb_classifier = XGBClassifier(random_state=state)

# Define hyperparameter space for each classifier
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
   
}

gb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [1,2, 3, 5, 7],
   
}

xgb_params = {
    'n_estimators': [100, 200, 250, 300],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [1,2, 3, 5, 7],
    "scale_pos_weight" : [1,2, 3, 5, 7],
}

In [10]:
# Create GridSearchCV objects for each classifier
rf_grid_search = GridSearchCV(estimator=rf_classifier, param_grid=rf_params, cv=5, scoring='accuracy')
gb_grid_search = GridSearchCV(estimator=gb_classifier, param_grid=gb_params, cv=5, scoring='accuracy')
xgb_grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=xgb_params, cv=5, scoring='accuracy')

In [11]:
# Fit the GridSearchCV objects to your data
rf_grid_search.fit(X_train, y_train)
gb_grid_search.fit(X_train, y_train)
xgb_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=0, ...),
             param_grid={'learning_rate': [0.1, 0.05, 0.01],
                         'max_depth': [1, 2, 3, 5, 7],
                         'n_estimators': [100, 200, 250, 300],
                         'scale_pos_weight': [1, 2, 3, 5, 7]},
             scoring='accuracy')

In [12]:
# Get the best hyperparameters for each classifier
best_rf_params = rf_grid_search.best_params_
best_gb_params = gb_grid_search.best_params_
best_xgb_params = xgb_grid_search.best_params_

print("Best Random Forest params:", best_rf_params)
print("Best Gradient Boosting params:", best_gb_params)
print("Best XGBoost params:", best_xgb_params)

Best Random Forest params: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100}
Best Gradient Boosting params: {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 300}
Best XGBoost params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'scale_pos_weight': 7}


In [13]:
# Get the best classifier models with the best parameters
best_rf_model = rf_grid_search.best_estimator_
best_gb_model = gb_grid_search.best_estimator_
best_xgb_model = xgb_grid_search.best_estimator_

In [14]:
best_rf_accuracy = best_rf_model.score(X_test, y_test)
best_gb_accuracy = best_gb_model.score(X_test, y_test)
best_xgb_accuracy = best_xgb_model.score(X_test, y_test)

print("Best Random Forest Accuracy:", best_rf_accuracy)
print("Best Gradient Boosting Accuracy:", best_gb_accuracy)
print("Best XGBoost Accuracy:", best_xgb_accuracy)

Best Random Forest Accuracy: 0.8629032258064516
Best Gradient Boosting Accuracy: 0.8870967741935484
Best XGBoost Accuracy: 0.8951612903225806


In [15]:
stack = make_pipeline(StackingCVRegressor(regressors=(best_rf_model, best_gb_model),
                                          meta_regressor=best_xgb_model,
                                          use_features_in_secondary=True,
                                          random_state=state))

In [16]:
stack.fit(X_train, y_train)

Pipeline(steps=[('stackingcvregressor',
                 StackingCVRegressor(meta_regressor=XGBClassifier(base_score=None,
                                                                  booster=None,
                                                                  callbacks=None,
                                                                  colsample_bylevel=None,
                                                                  colsample_bynode=None,
                                                                  colsample_bytree=None,
                                                                  early_stopping_rounds=None,
                                                                  enable_categorical=False,
                                                                  eval_metric=None,
                                                                  feature_types=None,
                                                                  gamma=None,
                                                                  gpu_id=None,
                                                                  grow_policy=None,
                                                                  importance_type=None,
                                                                  in...
                                                                  max_leaves=None,
                                                                  min_child_weight=None,
                                                                  missing=nan,
                                                                  monotone_constraints=None,
                                                                  n_estimators=200,
                                                                  n_jobs=None,
                                                                  num_parallel_tree=None,
                                                                  predictor=None,
                                                                  random_state=0, ...),
                                     random_state=0,
                                     regressors=(RandomForestClassifier(min_samples_split=5,
                                                                        random_state=0),
                                                 GradientBoostingClassifier(max_depth=1,
                                                                            n_estimators=300,
                                                                            random_state=0)),
                                     use_features_in_secondary=True))])

In [17]:
stack_accuracy_train = stack.score(X_train, y_train)
print("Stack Accuracy train:", stack_accuracy_train)

Stack Accuracy train: 1.0


In [18]:
stack_accuracy_test = stack.score(X_test, y_test)
print("Stack Accuracy test:", stack_accuracy_test)

Stack Accuracy test: 0.22012578616352185


The difference between the train and test is significant. That means that data is overfitting.

In [23]:
Kfold = KFold(n_splits = 10)
results_K = cross_val_score(stack, X, y, cv = Kfold)
print("KFold - Accuracy: {0}% ({1})%)".format(results_K.mean()*100, results_K.std()*100))

KFold - Accuracy: 55.893154383348474% (25.490826040868086)%)


**Even after the cross-validation the results showed low accuracy. This notebook was not considered for submission, but show how stacking of different models can be performed in a simple way.**